In [ ]:
import pickle
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

: 

In [ ]:
data = pd.read_pickle("bayesian array")
nodes = pd.read_pickle("bayesian nodes")

: 

In [ ]:
for i in range(1617):
    if data[0][i]==0:
        print('found true zero')

: 

In [ ]:
len(data[0])

: 

In [ ]:
def r_pow(x, n, d):
    """
    Compute x to the power of n/d (not reduced to lowest
    expression) with the correct function real domains.
    
    ARGS:
        x (int,float,array): base
        n (int)            : exponent numerator
        d (int)            : exponent denominator
        
    RETURNS:
        x to the power of n/d
    """
    
    # list to array
    if type(x) == list:
        x = np.array(x)
    # check inputs
    if type(n) != int or type(d) != int:
        raise Exception("Exponent numerator and denominator must be integers")
    # if denominator is zero
    if not d:
        raise Exception("Exponent denominator cannot be 0")
        
    # raise x to power of n
    X = x**n
    # even denominator
    if not d % 2:
        # domain is for X>=0 only
        if type(x) == np.ndarray:
            X[X<0] = np.nan
        elif X < 0:
            X = np.nan
        res = np.power(X, 1./d)
        return res
    # odd denominator
    else:
        # domain is all R
        res = np.power(np.abs(X), 1./d)
        res *= np.sign(X)
        return res

: 

In [ ]:
roots = [r_pow(data[0][i], 1, nodes[0][i]) for i in range(1617)] 

: 

In [ ]:
plt.loglog(nodes[0], data[0], lw=0, marker='x')
plt.ylim([0,0.5])

: 

In [ ]:
plt.plot(nodes[0], roots, lw=0, marker='x')
top20 = np.array([[nodes[0][i], roots[i]] for i in range(1617) if roots[i]>=np.percentile(roots,80)]).T
plt.plot(top20[0], top20[1], lw=0, marker='x')
bottom20 = np.array([[nodes[0][i], roots[i]] for i in range(1617) if roots[i]<np.percentile(roots,20)]).T
plt.plot(bottom20[0], bottom20[1], lw=0, marker='x')


: 

: 

In [ ]:
bottom20_i = np.array([i for i in range(1617) if roots[i]<np.percentile(roots,20)]).T
top20_i = np.array([i for i in range(1617) if roots[i]>=np.percentile(roots,80)]).T

print(bottom20_i)
print(top20_i)

: 

In [ ]:
nwks_list2 = pd.read_csv("nwks_list2")
print(nwks_list2.iloc[0])
cwd = os.getcwd() 
print("Current working directory:", cwd) 


def mega_file_reader2(removal = "random", adj_list = ["taro.txt"]):
    #, file_path = "C:\\Users\\jj\Downloads\\GitHub\small-perc\\pholme_networks" )
    if removal == "random":
        remove_bool = False
    elif removal == "attack":
        remove_bool = True
        
    for file_name in adj_list:

        file = open(file_name, "r")
        #file = open(os.path.join(file_path,file_name), "r")
        content=file.readlines()

        if len(content) == 0:
            file.close()
        # identify type of file (biadjacency matrix, edge list)
            # edge list
        elif len(content[0]) == 4 and (len(content) == 1 or len(content) > 2):
            print('edge file')
            if nodecount_edge(file_name) > 100:
                file.close()
            else:
                node_list = []
                edge_list = np.empty(len(content), dtype=object)
                for i in range(len(content)):
                    edge = content[i].strip()
                    edge = edge.split(" ")
                    edge_list[i] = np.zeros(2)
                    edge_list[i][0] = int(edge[0])
                    edge_list[i][1] = int(edge[1])
                    for j in range(2):
                        node_list.append(int(edge[j]))
                n = max(node_list) + 1
                adj = np.zeros((n, n))

                for k in range(len(edge_list)):
                    adj[int(edge_list[k][0]), int(edge_list[k][1])] = 1
                    adj[int(edge_list[k][1]), int(edge_list[k][0])] = 1

            #biadjacency matrix
        elif len(content[0]) > 4 or (len(content[0]) == 4 and len(content) == 2):
            print("biadj file")
            if nodecount_bi(file_name) > 100:
                file.close()
            else:
                edge_list = np.empty(len(content), dtype=object)
                # edge1 = content[0].strip()
                # edge1 = edge1.split("\t")
                k = len(content[0].strip().split("\t"))
                n = len(content) + k
                adj = np.zeros((len(content), k))
                big_adj = np.zeros((n, n))

                for i in range(len(content)):
                    edge = content[i].strip()
                    edge = edge.split("\t")
                    print("edge")
                    print(edge)

                    for j in range(len(edge)):
                        e = int(float(edge[j]))
                        if e >= 1:
                            adj[i, j] = 1
                            big_adj[i, j + len(content)] = 1
                            big_adj[j + len(content), i] = 1
                        else:
                            adj[i, j] = 0
                G_0 = nx.from_numpy_array(big_adj)
                # G_0 = nx.algorithms.bipartite.matrix.from_biadjacency_matrix(adj, create_using=None)
                G = G_0.copy()
                nx.draw(G)

                p = len(edge_list) / scipy.special.comb(n, 2)

                fig = plot_graphs(numbers_of_nodes=[n], edge_probabilities=[p],
                                  graph_types=['ER'], remove_strategies=["random"],
                                  performance='relative LCC', num_trials=100,
                                  smooth_end=False, forbidden_values=[], fdict=fvals, lcc_method_main="alice",
                                  savefig='')
                fig.savefig(file_name[:-4] + ".png")

                x_array = np.arange(0, n) / n
                averaged_data = np.zeros(n)
                for j_2 in range(100):
                    G = G_0.copy()
                    # print(list(G.nodes()), "nodes")
                    data_array = np.zeros(n, dtype=float)
                    for i_2 in range(n):
                        print(G.number_of_nodes(), "g size before")
                        data_array[i_2] = len(max(nx.connected_components(G), key=len)) / (n - i_2)
                        # find a node to remove
                        if removal == "random":
                            if G.number_of_nodes() != 0:
                                v = choice(list(G.nodes()))
                                G.remove_node(v)
                                # print(v)
                        elif removal == "attack":
                            if G.number_of_nodes() != 0:
                                v = sorted(G.degree, key=lambda x: x[1], reverse=True)[0][0]
                                G.remove_node(v)
                    averaged_data += data_array
                #print(x_array, "xarray")
                averaged_data /= 100
                #print(averaged_data, "y")

                fin = finiteTheory.relSCurve(p,n,attack=remove_bool, fdict=fvals,pdict=pvals,lcc_method_relS="pmult",
                                       executable_path = "C:\\Users\\jj\\Downloads\\GitHub\\small-perc\\p-recursion.exe")
                
                file.close()

    return (averaged_data, fin)

y_array_b = np.zeros(len(bottom20_i))
y_array_t = np.zeros(len(top20_i))

import os

mega_file_reader2(removal = "random", adj_list = ["C:\\Users\\jj\Downloads\\GitHub\small-perc\\pholme_networks\taro.txt"])

for i_b in bottom20_i:
    #print("file1",nwks_list2.iloc[i_b][1])
    file = nwks_list2.iloc[i_b][1]
    #file2 = file2.replace("C:\\Users\\jj\Downloads\\GitHub\small-perc\\pholme_networks", '')
    #print("file2",file)
    #file = file[60:]
    #print("file3",file2)
    values = mega_file_reader2(removal = "random", adj_list = [file])
    y = ((values[0]- values[1]) ** 2).mean()
    y_array_b[i_b] = y

for i_t in top20_i:
    values = mega_file_reader2(removal = "random", adj_list = [nwks_list2.iloc[i_t]])
    y = ((values[0]- values[1]) ** 2).mean()
    y_array_t[i_t] = y

plt.plot(top20[0],y_array_t, marker = "x", color = "red")
plt.plot(bottom20[0],y_array_b, marker = "x", color = "green")